# Panel Regression - Firm Characteristics

A panel regression is a suitable regression method for analyzing our data, especially since we have quarterly data for multiple companies over a long period of time. A panel regression model accounts for both within-entity and between-entity variations, making it a useful tool to analyze data with a time series and cross-sectional dimension.  
With panel data, we can control for individual-level characteristics that may affect forecast accuracy by including fixed effects for each company or industry. We can also account for time-specific factors that may affect forecast accuracy by including time fixed effects or time-varying covariates.

### Random Effects Panel Regression

In [248]:
import pandas as pd
import numpy as np
import datetime as dt
import sklearn
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects


Data

In [249]:
df = pd.read_csv("Dataframes/economic_story_regression.csv")
df["Date"] = pd.to_datetime(df["Date"])
df

,Instrument,Date,Earnings Per Share - Actual Surprise AbsVals,GICS Industry Group Name,Earnings Per Share – Coefficient of Variation AbsVals,Market Capitalization,Revenue - Actual,Number of Analysts,Recommendation - Mean (1-5).1,Price Target - Standard Deviation,3 Month Total Return,loss firm status,CBOE Crude Oil ETF Volatility Index,90-Day AA Financial Commercial Paper Interest Rate,Inflation Risk Premium,"University of Michigan: Consumer Sentiment, Index 1966:Q1=100",Unemployment Rate
0,AVY.N,2013-01-01,2.499631,Materials,-0.183599,-0.391005,-0.375130,-1.456340,3.000,-0.371337,0.433789,-1,-1.005294,-0.788065,-0.374525,-0.744622,1.362950
1,AVY.N,2013-04-01,1.247607,Materials,-0.233233,-0.383696,-0.377964,-1.456340,3.000,-0.313057,1.321869,-1,-0.945945,-0.810075,-0.361500,-0.334332,1.253195
2,AVY.N,2013-07-01,0.726582,Materials,-0.248775,-0.384120,-0.373419,-1.319027,2.875,-0.297757,-0.268964,-1,-0.905655,-0.828005,0.937254,-0.342537,1.088562
3,AVY.N,2013-10-01,2.207725,Materials,-0.210298,-0.383994,-0.377453,-1.456340,2.875,-0.331666,-0.103207,-1,-1.183895,-0.827825,0.661257,-0.722740,0.923930
4,AVY.N,2014-01-01,0.904623,Materials,-0.196835,-0.378999,-0.370730,-1.319027,2.625,-0.332724,0.786515,-1,-1.195206,-0.827407,0.958285,-0.394508,0.777590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19196,POOL.OQ,2021-10-01,2.901092,Retailing,-0.208403,-0.275230,-0.385406,-1.319027,2.300,1.529043,-0.602611,-1,0.303321,-0.813038,-0.083766,-1.299883,-0.576057
19197,POOL.OQ,2022-01-01,3.720063,Retailing,-0.194778,-0.231634,-0.417395,-1.319027,2.300,1.167079,1.738654,-1,0.920000,-0.490075,-0.105142,-1.855143,-0.795567
19198,POOL.OQ,2022-04-01,3.565072,Retailing,-0.089162,-0.278978,-0.385304,-1.181715,2.300,1.014923,-1.919184,-1,0.777108,0.426826,1.337373,-2.287316,-0.905322
19199,POOL.OQ,2022-07-01,0.917490,Retailing,-0.183610,-0.303046,-0.330569,-1.181715,2.000,1.809340,-1.689192,-1,0.739057,1.902525,0.424584,-2.432286,-0.923614


#### Fixed Effects

In a fixed effects panel regression, the individual-specific effects are modeled as fixed variables that do not vary across time. This means that the coefficients of the independent variables are estimated based on the within-entity variation in the data, which eliminates the effect of time-invariant unobserved heterogeneity.

Fixed effects models are useful when there are time-invariant unobserved variables that may affect the dependent variable, but are not included in the model. By modeling the individual-specific effects as fixed variables, fixed effects models can control for this unobserved heterogeneity and estimate the coefficients of the independent variables based on the within-entity variation, which provides more efficient estimates of the coefficients.

One limitation of fixed effects models is that they do not allow for testing the effect of time-invariant variables on the dependent variable. In addition, fixed effects models may suffer from the incidental parameter problem, which may lead to biased estimates of the coefficients of the independent variables in the presence of a large number of fixed effects.

# Fixed Effects Model

In [250]:
df = df.dropna()
df.set_index(['Instrument', 'Date'], inplace=True)
df

Earnings Per Share - Actual Surprise AbsVals  \
Instrument Date                                                       
AVY.N      2013-01-01                                      2.499631   
           2013-04-01                                      1.247607   
           2013-07-01                                      0.726582   
           2013-10-01                                      2.207725   
           2014-01-01                                      0.904623   
...                                                             ...   
POOL.OQ    2021-10-01                                      2.901092   
           2022-01-01                                      3.720063   
           2022-04-01                                      3.565072   
           2022-07-01                                      0.917490   
           2022-10-01                                      1.647889   

                      GICS Industry Group Name  \
Instrument Date                                  
AVY.N      2013-01-01                Materials   
           2013-04-01                Materials   
           2013-07-01                Materials   
           2013-10-01                Materials   
           2014-01-01                Materials   
...                                        ...   
POOL.OQ    2021-10-01                Retailing   
           2022-01-01                Retailing   
           2022-04-01                Retailing   
           2022-07-01                Retailing   
           2022-10-01                Retailing   

                       Earnings Per Share – Coefficient of Variation AbsVals  \
Instrument Date                                                                
AVY.N      2013-01-01                                          -0.183599       
           2013-04-01                                          -0.233233       
           2013-07-01                                          -0.248775       
           2013-10-01                                          -0.210298       
           2014-01-01                                          -0.196835       
...                                                                  ...       
POOL.OQ    2021-10-01                                          -0.208403       
           2022-01-01                                          -0.194778       
           2022-04-01                                          -0.089162       
           2022-07-01                                          -0.183610       
           2022-10-01                                          -0.176745       

                       Market Capitalization  Revenue - Actual  \
Instrument Date                                                  
AVY.N      2013-01-01              -0.391005         -0.375130   
           2013-04-01              -0.383696         -0.377964   
           2013-07-01              -0.384120         -0.373419   
           2013-10-01              -0.383994         -0.377453   
           2014-01-01              -0.378999         -0.370730   
...                                      ...               ...   
POOL.OQ    2021-10-01              -0.275230         -0.385406   
           2022-01-01              -0.231634         -0.417395   
           2022-04-01              -0.278978         -0.385304   
           2022-07-01              -0.303046         -0.330569   
           2022-10-01              -0.315123         -0.368055   

                       Number of Analysts  Recommendation - Mean (1-5).1  \
Instrument Date                                                            
AVY.N      2013-01-01           -1.456340                          3.000   
           2013-04-01           -1.456340                          3.000   
           2013-07-01           -1.319027                          2.875   
           2013-10-01           -1.456340                          2.875   
           2014-01-01           -1.319027                          2.625   
...             

In [251]:
y = df["Earnings Per Share - Actual Surprise AbsVals"]
X = df.loc[:, "Earnings Per Share – Coefficient of Variation AbsVals":]
# perform the fixed effects panel regression, setting check_rank=False
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True, drop_absorbed=True, check_rank=False)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary


C:\Users\alexa\AppData\Local\Temp\ipykernel_18340\597912754.py:7: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

CBOE Crude Oil ETF Volatility Index, 90-Day AA Financial Commercial Paper Interest Rate, Inflation Risk Premium, University of Michigan: Consumer Sentiment, Index 1966:Q1=100, Unemployment Rate

  fixed_effects_results = fixed_effects_model.fit()


Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.0655
Estimator:,PanelOLS,R-squared (Between):,0.4327
No. Observations:,18152,R-squared (Within):,0.0752
Date:,"Wed, Mar 08 2023",R-squared (Overall):,0.3845
Time:,16:11:39,Log-likelihood,-2.32e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,154.30
Entities:,501,P-value,0.0000
Avg Obs:,36.232,Distribution:,"F(8,17604)"
Min Obs:,1.0000,,
Max Obs:,40.000,F-statistic (robust):,154.30


# Without covid

In [252]:
df = pd.read_csv("Dataframes/economic_story_regression.csv")
df["Date"] = pd.to_datetime(df["Date"])

In [253]:
covid_start = pd.to_datetime("2020-01-01")

df_nocovid = df[df["Date"] < covid_start]
df_nocovid

,Instrument,Date,Earnings Per Share - Actual Surprise AbsVals,GICS Industry Group Name,Earnings Per Share – Coefficient of Variation AbsVals,Market Capitalization,Revenue - Actual,Number of Analysts,Recommendation - Mean (1-5).1,Price Target - Standard Deviation,3 Month Total Return,loss firm status,CBOE Crude Oil ETF Volatility Index,90-Day AA Financial Commercial Paper Interest Rate,Inflation Risk Premium,"University of Michigan: Consumer Sentiment, Index 1966:Q1=100",Unemployment Rate
0,AVY.N,2013-01-01,2.499631,Materials,-0.183599,-0.391005,-0.375130,-1.456340,3.00000,-0.371337,0.433789,-1,-1.005294,-0.788065,-0.374525,-0.744622,1.362950
1,AVY.N,2013-04-01,1.247607,Materials,-0.233233,-0.383696,-0.377964,-1.456340,3.00000,-0.313057,1.321869,-1,-0.945945,-0.810075,-0.361500,-0.334332,1.253195
2,AVY.N,2013-07-01,0.726582,Materials,-0.248775,-0.384120,-0.373419,-1.319027,2.87500,-0.297757,-0.268964,-1,-0.905655,-0.828005,0.937254,-0.342537,1.088562
3,AVY.N,2013-10-01,2.207725,Materials,-0.210298,-0.383994,-0.377453,-1.456340,2.87500,-0.331666,-0.103207,-1,-1.183895,-0.827825,0.661257,-0.722740,0.923930
4,AVY.N,2014-01-01,0.904623,Materials,-0.196835,-0.378999,-0.370730,-1.319027,2.62500,-0.332724,0.786515,-1,-1.195206,-0.827407,0.958285,-0.394508,0.777590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19184,POOL.OQ,2018-10-01,1.754231,Retailing,-0.235846,-0.363606,-0.436479,-1.456340,2.33333,0.192830,0.422311,-1,0.128630,1.550308,0.775160,1.016893,-0.777274
19185,POOL.OQ,2019-01-01,2.089639,Retailing,-0.085517,-0.369833,-0.459306,-1.319027,2.42857,0.052626,-0.966003,-1,-0.271228,1.542205,-1.885055,0.716013,-0.758982
19186,POOL.OQ,2019-04-01,3.181257,Retailing,-0.075237,-0.365303,-0.454679,-1.456340,2.50000,-0.054752,0.476689,-1,-0.324585,1.425644,-3.108622,1.044246,-0.887029
19187,POOL.OQ,2019-07-01,0.177309,Retailing,-0.226468,-0.356538,-0.410096,-1.319027,2.14286,0.182751,0.794719,-1,-0.212384,1.128836,-0.881147,0.661308,-0.887029


In [254]:
df_nocovid['Instrument'] = df_nocovid['Instrument'].astype('category')
df_nocovid['Date'] = pd.to_datetime(df_nocovid['Date'])

# set the index to be the time variable and the cross-sectional variable
df_nocovid.set_index(['Instrument', 'Date'], inplace=True)

C:\Users\alexa\AppData\Local\Temp\ipykernel_18340\876088522.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nocovid['Instrument'] = df_nocovid['Instrument'].astype('category')
C:\Users\alexa\AppData\Local\Temp\ipykernel_18340\876088522.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nocovid['Date'] = pd.to_datetime(df_nocovid['Date'])


In [255]:
y = df_nocovid["Earnings Per Share - Actual Surprise AbsVals"]
X = df_nocovid.loc[:, "Earnings Per Share – Coefficient of Variation AbsVals":]
# perform the fixed effects panel regression, setting check_rank=False
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True, drop_absorbed=True, check_rank=False)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary


C:\Users\alexa\mambaforge\envs\AnalystForecast\Lib\site-packages\linearmodels\panel\model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
C:\Users\alexa\AppData\Local\Temp\ipykernel_18340\2560714564.py:7: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

CBOE Crude Oil ETF Volatility Index, 90-Day AA Financial Commercial Paper Interest Rate, Inflation Risk Premium, University of Michigan: Consumer Sentiment, Index 1966:Q1=100, Unemployment Rate

  fixed_effects_results = fixed_effects_model.fit()


Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.0544
Estimator:,PanelOLS,R-squared (Between):,0.4719
No. Observations:,12713,R-squared (Within):,0.0556
Date:,"Wed, Mar 08 2023",R-squared (Overall):,0.4126
Time:,16:11:40,Log-likelihood,-1.563e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,87.788
Entities:,498,P-value,0.0000
Avg Obs:,25.528,Distribution:,"F(8,12218)"
Min Obs:,0.0000,,
Max Obs:,28.000,F-statistic (robust):,87.788
